# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/cities.csv")
cities


,Unnamed: 0,Date,City,Country,Lat,Long,Humid,Max Temp,Wind,Cloudy
0,0,1594597193,Hithadhoo,MV,-0.60,73.08,72,84.83,9.19,71
1,1,1594597235,Longyearbyen,SJ,78.22,15.64,77,44.89,1.39,86
2,2,1594596926,Mataura,NZ,-46.19,168.86,86,42.94,3.98,86
3,3,1594597205,Port Alfred,ZA,-33.59,26.89,36,58.23,10.20,0
4,4,1594596977,Ilulissat,GL,69.22,-51.10,70,48.20,3.36,23
...,...,...,...,...,...,...,...,...,...,...
560,560,1594597218,Ust-Kuyga,RU,70.02,135.60,32,78.85,4.25,38
561,561,1594597752,Dagana,MR,16.52,-15.51,83,82.40,2.24,96
562,562,1594597239,Ürümqi,CN,43.80,87.60,49,69.80,8.95,99
563,563,1594597251,Nhulunbuy,AU,-12.23,136.77,53,77.00,16.11,96


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure(zoom_level=1,center=(20,30))
locations = cities[["Lat","Long"]]
humidity = cities["Humid"]

In [6]:
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity = 40, point_radius=1)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weather_criteria = cities.loc[(cities["Max Temp"] >70) &
                              (cities["Max Temp"] < 80) &
                              (cities["Cloudy"]==0) &
                              (cities["Wind"] < 10)]
weather_criteria

,Unnamed: 0,Date,City,Country,Lat,Long,Humid,Max Temp,Wind,Cloudy
110,110,1594597650,Angoche,MZ,-16.23,39.91,83,70.81,6.20,0
157,157,1594597661,Capaci,IT,38.17,13.24,73,78.01,1.12,0
255,255,1594597685,Linkou,CN,45.30,130.28,69,74.26,3.29,0
289,289,1594597693,Waddān,LY,29.16,16.14,55,72.88,3.98,0
308,308,1594597697,Vila,PT,42.03,-8.16,60,75.99,5.82,0
380,380,1594597713,Mola di Bari,IT,41.06,17.09,65,75.99,1.01,0
410,410,1594597720,Paralímni,CY,35.04,33.98,94,77.00,3.36,0
503,503,1594597040,Poum,NC,-20.23,164.02,75,73.81,8.90,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weather_criteria.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,Date,City,Country,Lat,Long,Humid,Max Temp,Wind,Cloudy,Hotel Name
110,110,1594597650,Angoche,MZ,-16.23,39.91,83,70.81,6.20,0,
157,157,1594597661,Capaci,IT,38.17,13.24,73,78.01,1.12,0,
255,255,1594597685,Linkou,CN,45.30,130.28,69,74.26,3.29,0,
289,289,1594597693,Waddān,LY,29.16,16.14,55,72.88,3.98,0,
308,308,1594597697,Vila,PT,42.03,-8.16,60,75.99,5.82,0,
380,380,1594597713,Mola di Bari,IT,41.06,17.09,65,75.99,1.01,0,
410,410,1594597720,Paralímni,CY,35.04,33.98,94,77.00,3.36,0,
503,503,1594597040,Poum,NC,-20.23,164.02,75,73.81,8.90,0,


In [9]:
# From International Airport
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "keyword": "hotel"
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    long = row["Long"]
    params["location"] = f"{lat},{long}"
    response = requests.get(base_url, params=params).json()
    #pprint(response)
    results = response["results"]
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("No go, brah. Going to the next...")

No go, brah. Going to the next...
No go, brah. Going to the next...


In [10]:
#hotel_df

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))